## Team Members : Anmol Singh Suag, Sanuj Bhatia

We have chosen to work on <b> VAST Challenge 2015 </b> for the final project. The VAST Challenge 2015 has 2 Mini-challenges and 1 Grand Challenge. For the Homeworks 5-7, we would be working on the <b>Mini-Challenge 1</b>.

#### About MC1 of Vast Challenge 2015

* Mini-Challenge 1 of VAST Challenge 2015, provides location tracking information of the paying visitors of a theme park for 3 consecutive days. This data has been made possible as all paying visitors are made to use the park app while they are in the park.

* The dataset includes the following columns for 3 days.
    ID : The visitor ID of the paid park visitor
    Timestamp : The time of activity
    Activity Type : Check-in or Movement
    X : The X-Coordinate of location
    Y : The Y-Coordinate of location

* Data for every visitor is provided for each second, but isn't tracked after the visitor checks into a ride at the theme work. We assume the visitor to be inside the the ride until he appears back on the location grid for movement tracking.

* Users using the same app would have the same ID if they visit for more than one day.

* The Mini-Challenge 1 asks us to analyse this data to identify:
    * Number of group types that visit the park
    * Size of these groups
    * Most frequently visited places by these groups
    * Observations about the groups from the data
    * Activity patterns of the group
    * Suggesting improvement in the park
    * Anomalies in the park over the 3 days

This is a very interesting and complex problem-set and we had not worked with spatio-temporal data-sets before. Hence, we chose this VAST Challenge to augment our understanding about the same and use Machine Learning algorithms to solve this challenge.

In [1]:
#Importing commonly used ML Libraries
from bokeh.io import output_notebook, show
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import math
import copy
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from random import randint

In [2]:
output_notebook()

Loading BokehJS ...

### Selecting similar data-set

* For Homework 5, we had to search for a data-set similar to the one provided for MC1 of VAST Challenge 2015.

* After searching through myriad data-set repositories, we found 2 very similar data-sets at stanford's data repository. These are 'BrightKite' and 'Gowalla'.
    * BrightKite Data-set Link : https://snap.stanford.edu/data/loc-brightkite.html
    * Gowalla Data-set Link : https://snap.stanford.edu/data/loc-gowalla.html

* As 'BrightKite' and 'Gowalla' have the same structure, we chose to work on 'BrightKite' as it has comparatively smaller number of rows to process for our local machines and would serve as a good enough data-set to compare with MC1 data-set. 

* Brightkite was once a location-based social network where users could share their location by checking-in. The data has been gathered using their public API for around 58,000 users over a period of 2 years.

* The data contains over 4.5 million rows, with each row of the following format:
    * user_ID
    * timestamp
    * latitude 
    * longitude 
    * location_ID

* Each user has multiple check-ins at various locations identified by the latitude and longitude. For this assignment, we have taken the first 2000 users of the network (around 700k rows) as the time to run analyses on more data is too long.

* Using the User_ID, timestamp, latitude and longitude, we can simulate the data-set for the Mini-Challenge 1.

* 'BrightKite' data-set is different from the data-set of MC1 in the sense that it doesn't provide per second data and only provides check-in data. Our approach for the solution focuses on check-in counts and times, hence this can serve our purpose very well.

* Below is a small glimpse of the data set.

In [3]:
df = pd.read_table('Brightkite_totalCheckins.txt', header=None, names=['id', 'time', 'latitude', 'longitude', 'location_id'], usecols=['id', 'time', 'latitude', 'longitude'], nrows=713652, parse_dates=[1], infer_datetime_format=True)
df.head()

,id,time,latitude,longitude
0,0,2010-10-17 01:48:53,39.747652,-104.992510
1,0,2010-10-16 06:02:04,39.891383,-105.070814
2,0,2010-10-16 03:48:54,39.891077,-105.068532
3,0,2010-10-14 18:25:51,39.750469,-104.999073
4,0,2010-10-14 00:21:47,39.752713,-104.996337


### Data Preparation
1. We round the latitude and longitude to 2 decimal places to account for slight variation in checking-in at the same location. Second decimal place comes to anywhere from 700m to 1.1km variation, depending upon the exact location. This way, we can recognise unique locations throughout the data. 

2. The location ID column was removed because it was a hash value combining the 6 decimal place latitude and longitudes, and is not helpful in actually recognising unique locations after rounding them off.


In [37]:
df['latitude'] = round(df['latitude'], 2)
df['longitude'] = round(df['longitude'], 2)
df.head(5)

,id,time,latitude,longitude
0,0,2010-10-17 01:48:53,39.75,-104.99
1,0,2010-10-16 06:02:04,39.89,-105.07
2,0,2010-10-16 03:48:54,39.89,-105.07
3,0,2010-10-14 18:25:51,39.75,-105.00
4,0,2010-10-14 00:21:47,39.75,-105.00


### Solution Approach 1

* We mark unique locations as a conjugation of the truncated latitude and longitude values. These conjugated location identifiers would help us in clustering the users that have checked-in similar locations.

* Then we create a 2-D matrix of user-Ids and these location identifiers. The cells in the matrix contain the number of times the user has checked-in at that specific location.

* We aim to cluster the users based on this check-in matrix. Users that have similar check-ins and similar number of check-ins at different locations can be assumed to be a part of a group.

* Note that we haven't considered the time of check-ins in this approach. We will use the time-stamps to validate the results of our clustering algorithm.

In [5]:
unique_locations = {}
id_visits = {}
for i in range(713652):
    loc_id = str(df['latitude'][i])+str(df['longitude'][i])
    unique_locations[loc_id] = True

In [6]:
print (len(unique_locations))
unique_locations = list(unique_locations)

43028


In [35]:
users = np.zeros(shape=(2000,43028))
for i in range(713652):
    userId = df['id'][i]
    locIndex = unique_locations.index(str(df['latitude'][i])+str(df['longitude'][i]))
    users[userId][locIndex]+=1

### Machine Learning Algorithm Used

* For this approach, we use KMeans Clustering algorithm. Kmeans is an unsupervised and robust learning algorithm that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster.

* The number of clusters in KMeans is fixed before-hand. After analysing the nature of data and probable number of groups that could be identified, we ran the algorithm on our user-checkin matrix with different number of cluster values. After many test runs, we finalised the cluster count value as 1000 to be a good test guess.

* We would be using KMeans various times during the solution of this Mini-Challenge. In the current appraoch we use KMeans on userId and check-in count matrix, in the later approaches we would use KMeans to cluster users with similar time of check-ins at the same locations.

In [36]:
kmeans = KMeans(n_clusters=1000, n_jobs=-1, n_init=7).fit(users)

In [42]:
# Separating Groups
kmeans_groups = kmeans.labels_
groups = {}
for i in range(len(kmeans_groups)):
    if kmeans_groups[i] in groups:
        groups[kmeans_groups[i]].append(i)
    else:
        groups[kmeans_groups[i]] = [i]


* We created a dictionary of 'Cluster Label' to 'UserIds'. The users belonging to the same cluster are hoped to be a part of the same group.

* We have found 20 decently sized groups, but many other clusters had a single user. This is possible because:
    * We chose a smaller subset of the full data.
    * The time range of check-in spans across years.
    * This is a social-network data, users are distributed far apart
    
* We hope this approach to work well with the actual data-set of MC1 because that data-set is of a small location and the probability of groups to check-in at the same place is higher. Secondly, the MC1 data-set is just for 3 days as compared to BrightKite, which spans over 2 years.

* We now validate one such group in BrightKite data that has been identified through KMeans.

In [63]:
#Looking into cluster #655
print ('Size of Group 655: ',len(groups[655]))
print ('Members in the group (user IDs): ', groups[655])

presenceDict={}
for i in range(len(groups[655])):
    userId=groups[655][i]
    userDF = df[df['id'] == userId]
    for j in range(len(userDF)):
        presenceID = str(userDF.iloc[j,1].date()) + '_' + str(userDF.iloc[j,2]) + '_' + str(userDF.iloc[j,3])
        if presenceID in presenceDict:
            if userId in presenceDict[presenceID]:
                pass
            else:
                presenceDict[presenceID].append(userId)
        else:
            presenceDict[presenceID] = [userId]

for key, value in presenceDict.items():
    if len(value) > 1:
        formatKeys = key.split('_')
        print('Date: ' + formatKeys[0] + ', Lat: ' + formatKeys[1] + ', Long: ' + formatKeys[2] + ', Users Checking in together: ', value)


Size of Group 655:  9
Members in the group (user IDs):  [82, 98, 976, 1019, 1025, 1044, 1088, 1325, 1580]
Date: 2009-09-05, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 98]
Date: 2008-10-25, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 1025, 1044, 1325]
Date: 2008-10-23, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 1325, 1580]
Date: 2008-10-21, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 1025, 1325]
Date: 2008-09-11, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 98]
Date: 2008-08-26, Lat: 39.74, Long: -104.98, Users Checking in together:  [82, 1088]
Date: 2009-05-23, Lat: 40.02, Long: -105.28, Users Checking in together:  [98, 1580]
Date: 2008-12-18, Lat: 39.74, Long: -104.98, Users Checking in together:  [98, 976, 1044]
Date: 2008-11-26, Lat: 39.74, Long: -104.98, Users Checking in together:  [98, 976]
Date: 2008-11-19, Lat: 39.74, Long: -104.98, Users Checking in together:  [98, 976, 1044]
Date: 2008-11-1

### Solution Validation

* We chose one of the clusters to validate if the userIds in that group were actually present together on the same date at the same location.

* We created a dictionary where the key is a conjugation of date,latitude and logitude and the values are the userIds that were present at that date on the same location.

* We have printed the results and we can see that it is actually a good group where the group members have checked-in together at the same locations on the same date!

### Future Approaches
* We plan to use KMeans Clustering algorithms on 2-D sparse matrices of UserIds vs Check-In times as well as UserIds vs unique spatio-temporal identifiers. We will compare the groups created in these approaches to our previous approach.

* Analysing the matrices, we can identify common locations visited by groups, frequency of check-ins at various locations, group check-in behaviour over a day/week/year as well as anomalies. By plotting the check-ins over time, we can identify the dates on which more check-ins were made vs dates of low check-ins. As of now we hope to have more check-ins on weekends as compared to weekdays, but this would be clearer after making proper visualisations like:
    * Bar Charts of frequency of check-ins over 7 day week
    * Line graphs of check-in counts over a year
    * Line graphs of check-in over the time of day

* For the groups, we plan to plot the trajectories of the users in increasing time order and hence validate that users in the same group have common/overlapping paths. 

* For visualising location check-in counts, we would make scatter plots of locations with their X and Y coordiantes. The size of scatter point would be proportional to the number of check-ins it had.

* By creating a histogram of group sizes vs number of groups of that size, we can identify what group sizes are more common. We expect that larger group sizes would be less common whereas smaller group sizes or individuals would be more common.

* We can also determine what locations are favored over a day by finding the percentage of check-ins for a particular location. We would use interactive scatter plots to visualise the percentage of checkins a location had for that day. 